#Assembly

Let's make two element model and assemble it. We split the previous one element model to 2 quadrilaterals, make assembly and solve it. Small modifications to functions, I think it's better that they don't allocate memory but do in place operations. 

**TODO**
- Tangent stiffness is calculated using forward finite difference. I think we should try ReverseDiffSparse for it's sparse matrix support, but I don't know how to use it. Or alternatively use FAD like before and assemble after linearization. It would be nice experiment to try linearization *after* assembly, would it work?
- Verify calculations using some well known FEM software.


Author: Jukka Aho

Email: <jukka.aho@kapsi.fi>

In [1]:
tic()

0x00000c95556709c2

In [2]:
type Node
    coords
end
type Element
    node_ids
end
elements = Dict()
nodes = Dict()

Dict{Any,Any} with 0 entries

In [3]:
ndim = 2
nnodes = 6

nodes[1] = Node([0, 1, 0])
nodes[2] = Node([5, 1, 0])
nodes[3] = Node([10, 1, 0])
nodes[4] = Node([0, 0, 0])
nodes[5] = Node([5, 0, 0])
nodes[6] = Node([10, 0, 0])
elements[1] = Element([4, 5, 2, 1])
elements[2] = Element([5, 6, 3, 2])

Element([5,6,3,2])

In [4]:
# Partial derivatives of bilinear Lagrange polynomials
dNdξ(ξ) = [[-(1-ξ[2])/4.0    -(1-ξ[1])/4.0],
           [ (1-ξ[2])/4.0    -(1+ξ[1])/4.0],
           [ (1+ξ[2])/4.0     (1+ξ[1])/4.0],
           [-(1+ξ[2])/4.0     (1-ξ[1])/4.0]]    

E = 90
ν = 0.25
μ = E/(2*(1+ν))
λ = E*ν/((1+ν)*(1-2*ν))
λ = 2*λ*μ/(λ + 2*μ)
μ, λ

(36.0,24.0)

In [5]:
function calculate_internal_energy!(X, u, Wint, dNdξ, λ, μ, dim=2)
    """Calculate internal energy for a single element.

    Parameters
    ----------
    X : array [dim x nodes]
    u : array [dim x nodes]
    dNdξ : shape function derivatives
    λ : float
    μ : float
    dim : integer, optinal

    Returns
    -------
    Nothing, this is inplace function
    
    """
    I = eye(dim)
    
    function J(ξ)
        Jᵀ = X*dNdξ(ξ)
        ∇N = inv(Jᵀ)*dNdξ(ξ)'
        ∇u = u*∇N'
        F = I + ∇u  # Deformation gradient
        E = 1/2*(∇u' + ∇u + ∇u'*∇u)  # Green-Lagrange strain tensor
        P = λ*trace(E)*I + 2*μ*E  # PK1 stress tensor
        S = F*P  # PK2 stress tensor
        return S*∇N*det(Jᵀ)
    end

    a = 1/sqrt(3)
    ipoints = [[-a -a], [a -a], [a a], [-a a]]
    iweights = [1 1 1 1]

    for m = 1:length(iweights)
        w = iweights[m]
        ξ = ipoints[m, :]
        Wint[:,:] += w*J(ξ)
    end

end

calculate_internal_energy! (generic function with 2 methods)

In [6]:
Wint = zeros(2, 4)
X = [0 0; 1 0; 1 1; 0 1]'
u = [0 0; 0 0; 1 0; 0 0]'
calculate_internal_energy!(X, u, Wint, dNdξ, λ, μ)
Wint

2x4 Array{Float64,2}:
 -60.6667  -1.33333  133.333  -71.3333
 -24.0     -8.0       35.0     -3.0   

In [7]:
function assemble!(u, R)
    """ Assemble global residual vector R = T - F
    """
    R[:] = 0.0
    u = reshape(u, ndim, nnodes)
    R = reshape(R, ndim, nnodes)
    
    Xe = zeros(ndim, 4)
    Winte = zeros(ndim, 4)
    
    # Internal forces, T
    for i=1:length(elements)  # loop through elements
        Xe[:,:] = 0.0  # FIXME: how to efficiently empty array?
        el = elements[i]
        nids = el.node_ids
        for i=1:length(nids)  # loop through nodes
            Xe[:,i] = nodes[nids[i]].coords[1:2]
        end
        Winte[:,:] = 0.0
        calculate_internal_energy!(Xe, u[:,nids], Winte, dNdξ, λ, μ)
        R[:,nids] += Winte
    end

    # External forces, F
    # T - F = T + (-F)
    R[2, 3] += 2  # Force to the tip of härveli

    u = reshape(u, ndim*nnodes)
    R = reshape(R, ndim*nnodes)
end

assemble! (generic function with 1 method)

In [8]:
R = zeros(ndim*nnodes)
u = [0 0; 0 0; 1 0; 0 0; 0 0; 0 0]'
#u = [0 0; 0 0; 1 0; 0 0]'
assemble!(reshape(u, ndim*nnodes), R)
R'

1x12 Array{Float64,2}:
 0.0  0.0  34.8853  16.2  140.715  78.6  …  -64.1013  -36.0  -111.499  -56.8

We should use ReverseDiffSparse because of it's sparse matrix support

In [9]:
# FIXME: I don't know how to get these working!
#using ReverseDiffSparse
#using ForwardDiff
#Kt! = forwarddiff_jacobian!(assemble!, Float64, fadtype=:dual, n=12, m=12)
#Kt!(reshape(u, 12), Kt)
#Kt

In [10]:
# So we go to plan B. FIXME: change this to analytical version

function Lin(f!, h=1.0e-6)

    function D!(x, J)
        J[:,:] = 0
        N = length(x)
        Δx = zeros(N)
        y = zeros(N)
        Δy = zeros(N)
        f!(x, y)  # Evaluate function f in x and store results to y
        for i=1:N
            Δx[:] = 0.0
            Δx[i] += h
            f!(x+Δx, Δy)  # Evaluate function f in x+Δx and store results to Δy
            J[i, :] = (Δy-y) / h
        end
    end

    return D!

end

Lin (generic function with 2 methods)

Handling homogeneous Dirichlet conditions, using elimination.

**INFO**: We could try something like this: http://www.code-aster.org/V2/doc/default/en/man_r/r3/r3.03.01.pdf

Here's an idea how to make a very simply elimination

In [11]:
fixed_dofs = integer(zeros(ndim, nnodes))
fixed_dofs[:,1] = fixed_dofs[:,4] = 1
fixed_dofs

2x6 Array{Int64,2}:
 1  0  0  1  0  0
 1  0  0  1  0  0

In [12]:
free_dofs = integer(ones(ndim, nnodes)) - fixed_dofs
free_dofs

2x6 Array{Int64,2}:
 0  1  1  0  1  1
 0  1  1  0  1  1

In [13]:
free_dofs = find(free_dofs)
free_dofs'

1x8 Array{Int64,2}:
 3  4  5  6  9  10  11  12

In [14]:
function solve!(u, free_dofs; max_iterations=10, eps=1.0e-7)
    ∇R = Lin(assemble!)
    Kt = zeros(ndim*nnodes, ndim*nnodes)
    R = zeros(size(u))
    println("Starting Newton iterations")
    for i=1:max_iterations
        print("Iteration ",i, ", ")
        R[:] = 0.0
        assemble!(u, R)  # Calculate internal energy in nodes and store results to R
        ∇R(u, Kt)  # Linearize residual in u and save result to Kt
        # Solve !
        du = Kt[free_dofs, free_dofs] \ -R[free_dofs]
        u[free_dofs] += du
        println("norm = ",norm(du))
        if norm(du) < eps
            println("Converged.")
            break
        end
    end
    return u
end

u = zeros(ndim*nnodes)
solve!(u, free_dofs)
u = reshape(u, ndim, nnodes)
u

Starting Newton iterations
Iteration 1, norm = 12.031651257897487
Iteration 2, norm = 3.141275068747564
Iteration 3, norm = 1.1530769747882408
Iteration 4, norm = 0.2665784114781298
Iteration 5, norm = 0.035146969229970175
Iteration 6, norm = 0.003125584242156337
Iteration 7, norm = 2.227371173806294e-6
Iteration 8, norm = 2.1397676611257455e-10
Converged.


2x6 Array{Float64,2}:
 0.0  -0.106192  -1.69115  0.0  -0.761252  -2.48604
 0.0  -2.10509   -6.00728  0.0  -1.89221   -5.5914 

In [15]:
toc()

elapsed time: 4.541729719 seconds


4.541729719